In [ ]:
import pandas as pd
from plotly import express as ex
from tqdm import trange
import os

# MJ2383 Computer Lab 3

This lab aims to provide an inside view on what a supply curve is and how it can be generated while using OSeMOSYS. Reminder, a supply curve is a graphical representation of the law of supply. It slopes upward because the quantity supplied increases as price increases, with other things constant.

In this lab, we'll be using OSeMOSYS, but we'll be running it in the background using this Jupyter Notebook to control the input data, run the model and extract and visualise the results.

If you click on the **Jupyter** logo in the top left-hand corner of the screen, you will see the folder structure containing the OSeMOSYS models used in this lab.

In the `model` folder, you will find subfolders containing OSeMOSYS models of increasing complexity. Each subfolder contains a data folder in which you see a number of CSV (comma-separated value) files which you can edit directly in the browser.  For example, you could edit the `CapitalCost` of technologies by editing the respective [CSV file](../edit/model/gas/data/CapitalCost.csv).

Running a model is a two step process. Firstly, you need to create an OSeMOSYS datafile by running the following:

    !otoole convert datapackage datafile model/gas temp.txt
    
Note you need the `!` prepended to the command (this tells the notebook to run this command using the shell rather than Python). Here we create datafile called `temp.txt` from the model data stored in `model/gas`.

After generating the OSeMOSYS datafile (in this case, called `temp.txt`) we then solve the model using `glpsol`, which is the open-source solver provided by the GNU Math Programming kit.

    !glpsol -d temp.txt -m osemosys.txt > osemosys.log
    
The results are then available in the `results` folder - see them [here](../tree/results). You should also check the `osemosys.log` file to ensure that the model ran correctly.  View it [here](../edit/osemosys.log).

In the stages below, we load data from the model inputs and results using a Python library called `pandas` and plot them in interactive charts.

## Contents

- [Stage 1](#Stage-1:-A-super-simple-supply-curve) - In this section, we expore a very simple model with a two-stage supply curve for natural gas to develop our economic interpretation of OSeMOSYS
- [Stage 2](#Stage-2:-Adding-Resources) - We add complexity, by increasing the number of steps in our supply curve by adding resources. We explore what difference this makes to the electricity price under different conditions.
- [Stage 3](#Stage-3:-A-tax-on-pollution) - We add an emissions penalty, imposing a tax upon CO2
- [Stage 4](#Stage-4:-Renewable-electricity-%22there's-no-such-thing-as-a-free-lunch%22) - We add renewable technologies to the electricity sector, whose marginal cost of generation is 0. However, this creates a demand for backup capacity. How does this affect the marginal price of electricity?

## Stage 1: A super simple supply curve

We start with the simplest supply cost curve you can imagine. In this simple OSeMOSYS model, there are two natural gas commodities (fuels) - imports of natural gas `GasImport`, and extraction `GasExtraction`. Both of these generate CO2, and feed a natural gas combined-cycle electricity generation plant `NGCC`. This produces secondary electricity `SEC_EL` which enters a transmission and distribution technology `TD` which produces final electricity `FEL`.

![](img/simple.svg)

(Note, this image is produced using the command `!otoole viz res model/gas/datapackage.json img/simple.svg`)

---

First, let's have a think about the supply curve in this system.

Shown on the left of the image, there are two natural gas resources. `GasExtraction` has a [maximum production capacity](../edit/model/gas/data/TotalAnnualMaxCapacity.csv) of 6 PJ/year at a [variable cost](../edit/model/gas/data/VariableCost.csv) of €8/PJ. However, `GasImport` has no upper bound, but a higher cost of €12/PJ.

Plotted, this looks rather uninspiring, but at least gives an impression of a supply curve for natural gas.

__Run the next cell to view the graph__


In [ ]:
gas_extraction = [["GasExtraction", x, 8] for x in range(7)]
gas_import = [["GasImport", x, 12] for x in range(6, 21)]
gas_supply_curve = pd.DataFrame(gas_extraction + gas_import, 
    columns=['name', 'supply', 'marginal_cost'])
fig = ex.line(gas_supply_curve, x='supply', y='marginal_cost', range_x=[0, 20], range_y=[0, 20], line_shape='vh')
fig.update_traces(mode="markers+lines")
fig.update_xaxes(showspikes=True)
fig.show()

In OSeMOSYS, supply of energy must equal demand for energy. As the quantity of energy demanded increases, energy supply increases accordingly. Given that our demand curve is a straight line (imagine a vertical line moving along the x-axis) the equilibrium point (where the lines cross) denotes the marginal cost of production and in this case is equal to the price.

Within most energy systems, there are multiple markets for different, related, energy commodities. Later, we will use our simple OSeMOSYS model to begin to understand the interactions between these different energy markets.

### Exercise 1: Manually computing the marginal cost of electricity

Think about how our simple supply curve for natural gas interacts with the demand for electricity. 

In our simple example, we demand a specific quantity of electricity. The electricity is produced by the `NGCC` technology which requires almost 2 units of gas to produce 1 unit of electricity.

**Now answer the quiz question on Canvas.**

### Marginal Prices

In OSeMOSYS, there is a key equation called a "balancing constraint". This ensures that energy is conserved in each timeslice. 

```ampl
s.t. EBa11_EnergyBalanceEachTS5{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR}: 
	Production[r,l,f,y] >= Demand[r,l,f,y] + Use[r,l,f,y] 
	+ sum{rr in REGION} Trade[r,rr,l,f,y] * TradeRoute[r,rr,f,y];
```

We can extract some useful information from this equation when we solve the optimisation problem to minimise costs.  These data are stored in the file `results/ProductionDual.csv`.

### Exercise 2: Understanding production

We'll now run OSeMOSYS from this Jupyter Notebook to compute the equilibrium price:

In [ ]:
def write_demand(model, value):
    """Write a ``value`` into the SpecifiedAnnualDemand file
    """
    demand = pd.DataFrame([['SIMPLICITY', 'FEL', 2014, value]], columns=["REGION","FUEL","YEAR","VALUE"])
    demand.to_csv(os.path.join(model, "data/SpecifiedAnnualDemand.csv"), index=False)

In the plot below, we can see the total production by fuel over the year.  

In [ ]:
demand = 7
write_demand("model/gas", demand)
!otoole convert datapackage datafile model/gas temp.txt
# Solve the model, writing the results into ./results
!glpsol -d temp.txt -m osemosys.txt > osemosys.log
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
ex.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY')

However, production differs across the year - in OSeMOSYS we use `TIMESLICES` to represent the fractions of the year in which different levels of demand exist. This approximates the average demand profile for electricity and other fuels across days and seasons. In our example model, we have defined 6 time-slices:

TIMESLICE | Description
---|---
ID | Intermediate day
IN | Intermediate night
SD | Summer day
SN | Summer night
WD | Winter day
WN | Winter night

In [ ]:
production = pd.read_csv('results/ProductionByTechnology.csv').groupby(by=['TIMESLICE', 'FUEL']).sum()
fig = ex.bar(production.reset_index(), x='FUEL', y='VALUE', color='TIMESLICE')
fig.update_layout(barmode='group')

**Now answer the canvas quiz**

### Exercise 3: Using OSeMOSYS to compute the marginal price

Below, we use a Python library called `pandas` to read the `ProductionDual` comma-separated values file. We extract the data point for `GAS` during the winter day (`WD`) timeslice.

In [ ]:
cost = pd.read_csv('results/ProductionDual.csv').set_index(['TIMESLICE', 'FUEL'])
cost['DUAL'] = cost['DUAL'] / (1.05**-0.5)
cost.loc[('WD', 'GAS'),'DUAL']

By running the cell, we see that the marginal cost of producing gas, is equal to the cost of importing natural gas: about €12/PJ.  Interesting.  That must mean that the model is importing gas, rather than extracting it (which is cheaper).

NB: The division of `(1.05**-0.5)` compounds the marginal value back to the base year`

**Answer the quiz on canvas**

Plotted onto the supply curve we plotted earlier, we see that the results from the OSeMOSYS model make sense. The marginal cost of gas production from the OSeMOSYS model sits on the supply cost curve of the two gas resources.

However, we only have one point for electricity. In the next stages, we will use the model to compute the marginal cost of electricity production.

In [ ]:
fig = ex.line(gas_supply_curve, x='supply', y='marginal_cost', range_x=[0, 20], range_y=[0, 30], line_shape='vh')

gas_demand = production.groupby(by='FUEL').sum().loc['GAS', 'VALUE']
elec_demand = production.groupby(by='FUEL').sum().loc['FEL', 'VALUE']

fig.add_shape( # add a vertical "demand" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=gas_demand, x1=gas_demand, xref="x", y0=0, y1=cost.loc[('WD', 'GAS'),'DUAL'])

fig.add_annotation( # add a text callout with arrow
    text="Marginal cost of gas", x=gas_demand, y=cost.loc[('WD', 'GAS'),'DUAL'], arrowhead=1, showarrow=True
)

fig.add_annotation( # add a text callout with arrow
    text="Marginal cost of electricity", x=elec_demand, y=cost.loc[('WD', 'FEL'),'DUAL'], arrowhead=1, showarrow=True
)

fig.add_shape( # add a vertical "demand" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=elec_demand, x1=elec_demand, xref="x", y0=0, y1=cost.loc[('WD', 'FEL'),'DUAL'])

fig.show()

### Notes

- The x-axis shows the supply quantity of energy
- The marginal cost of electricity is plotted for reference against the quantity of electricity supplied, and marginal cost of gas against the quantity of gas supplied

### Running the model to extract the marginal cost of production of electricity

We can automate the running of the OSeMOSYS model over multiple levels of demand for electricity. The following code loops over demands ranging from 1 to 100 in increments of 5 (line 3).  In line 5 we write the demand into the CSV file `SpecifiedAnnualDemand.csv`, create the modelfile and then solve the model. We then extract the results from the `results/ProductionDual.csv` file and store them in a list.  Finally, the function returns the list of results for further processing.

In [ ]:


def extract_result(param: str, production, cost):
    marginal = pd.read_csv('results/ProductionDual.csv'
                           ).set_index(['TIMESLICE', 'FUEL'])
    marg_ann = pd.read_csv('results/ProductionDualAnnual.csv'
                           ).set_index(['FUEL'])
    production = pd.read_csv('results/ProductionByTechnology.csv').groupby(by=['TIMESLICE', 'FUEL']).sum()
    try:
        demand = production.groupby(by='FUEL').sum().loc[param, 'VALUE']
    except KeyError:
        demand = 0
    try:
        value = marginal.loc[('WD', param), 'DUAL'] / (1.05**-0.5)
    except KeyError:
        try:
            value = marg_ann.loc[param, 'DUAL'] / (1.05**-0.5)
        except KeyError:
            value = 0
    observation = {'value': value, 'param': param, 'quantity': demand}  
    return observation

def run_model(path_to_model: str, start: int, stop: int, step=int):
    """Run the model for a range of demand values
    
    Returns
    -------
    List
        A list of dual values for GAS and FEL
    """
    results = []
    for dem in trange(start, stop, step):
        observation = {}
        write_demand(path_to_model, dem)
        !otoole convert datapackage datafile $path_to_model temp.txt
        !glpsol -d temp.txt -m osemosys.txt > osemosys.log
        results.append(extract_result('FEL', production, cost))
        results.append(extract_result('GAS', production, cost))
        results.append(extract_result('COA', production, cost))

    return results


In [ ]:
results = run_model("model/gas", 1, 20, 1)

In [ ]:
data = pd.DataFrame(results)
ex.line(data, x='quantity', y='value', facet_col='param', range_x=[0, 20], range_y=[0, 40], 
        line_shape='hv')

# Stage 2: Adding Resources

In this part of the lab, we add a number of resources to our model and explore how this affects the supply curve for electricity. We now move towards a slightly more realistic energy system than the simplified model we used to introduce the concepts in Stage 1.

![](img/gasmore.svg)

(Note, this image is produced using the command `!otoole viz res model/gas_more/datapackage.json img/simple.svg`)

As you can see from the figure above, this model contains multiple gas resources, plus imports, multiple coal resources, plus an import, and a new super-critical coal fired power station.

The prices and quantities of the resources are as follows:

Resource | Quantity available | Cost of extraction/import
---|---|---
GasExtraction | 10 | 8.0
GasExtraction2 | 15 | 10.0
GasExtraction3 | 30 | 11.0
GasImport | - | 20.0
CoalExtraction | 10 | 4.0
CoalExtraction2 | 15 | 5.0
CoalExtraction3 | 30 | 5.5
CoalImport | - | 12.0

These prices and quantities are defined in [`VariableCost.csv`](../edit/model/gas_more/data/VariableCost.csv) and [`TotalTechnologyModelPeriodActivityUpperLimit.csv`](../edit/model/gas_more/data/TotalTechnologyModelPeriodActivityUpperLimit.csv) respectively.

The gas-fired (`NGCC`) and coal-fired (`SCC`) power plants have the following characteristics:

Plant | Parameter | Value
---|---|---
NGCC | CapitalCost | 1100.0
SCC | CapitalCost | 1600.0
NGCC | FixedCost | 44.0
SCC | FixedCost | 56.0
NGCC | InputActivityRatio(GAS) | 1.992
SCC | InputActivityRatio(COA) | 2.120
NGCC | OutputActivityRatio(SEC_EL) | 1.0
SCC | OutputActivityRatio(SEC_EL) | 1.0

In [ ]:
results = run_model("model/gas_more", 1, 60, 2)

In [ ]:
data = pd.DataFrame(results)
ex.line(data, x='quantity', y='value', facet_col='param', 
        range_x=[0, 80], range_y=[0, 45], line_shape='hv')

We now have a much more complicated supply cost curve for all commodities. Four distinct steps are evident in the supply cost curve for gas (`GAS`), three/four for coal (`COA`), and seven in the the plot for final electricity (`FEL`). Each of these steps in the gas and coal plots correspond to one of the resources that we defined in this more complicated model. However, the supply cost curve for electricity is a function of the combination of the gas and coal cost curves, and depends on the blend of coal and gas used to supply electricity at each level of demand.

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
ex.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY')

We can confirm the outputs from the model run with maximum demand by viewing the production of the different technologies. In the plot above you can view the quantity of coal and gas extracted or imported, and the quantity of secondary electricity produced by the power plants and final electricity transmitted to meet the demands.

**Now, answer the quiz on canvas**

# Stage 3: A tax on pollution

In this stage, we'll now implement a financial penalty for emission of CO2.  Before proceeding, please **answer the quiz on canvas**.

# Stage 4: Renewable electricity "there's no such thing as a free lunch"

![](img/gassolarwind.svg)

In [ ]:
results = run_model("model/gas_solar_wind")

In [ ]:
data = pd.DataFrame(results)
ex.line(data, x='demand', y='value', color='param', range_x=[0,100], range_y=[0,50])

In [ ]:
!otoole convert datapackage datafile model/gas_solar_wind temp.txt
# Solve the model, writing the results into ./results
!glpsol -d temp.txt -m osemosys.txt 

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
ex.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY')

# Extension Tasks

If you finish during the lab, you might want to complete one or more of the following optional tasks:

- Return to stage 2 and add a wind or solar technology into the OSeMOSYS model stored in `model/gas_more/data`.  You'll need to edit the following files: 
  - `InputActivityRatio.csv`, 
  - `OutputActivityRatio.csv`, 
  - `OperationalLife.csv`, 
  - `ResidualCapacity.csv`, 
  - `TECHNOLOGY.csv`, 
  - `FixedCost.csv` and 
  - `VariableCost.csv`.
  
  Once you've implemented the renewable technology, observe what happens to the supply cost curve.